***
# 数据抓取：

> # 使用Python编写网络爬虫
***

王成军

wangchengjun@nju.edu.cn

计算传播网 http://computational-communication.com

# 需要解决的问题 

- 页面解析
- 获取Javascript隐藏源数据
- 自动翻页
- 自动登录
- 连接API接口


In [1]:
import requests
from bs4 import BeautifulSoup

- 一般的数据抓取，使用urllib2和beautifulsoup配合就可以了。
- 尤其是对于翻页时url出现规则变化的网页，只需要处理规则化的url就可以了。
- 以简单的例子是抓取天涯论坛上关于某一个关键词的帖子。
    - 在天涯论坛，关于雾霾的帖子的第一页是：
http://bbs.tianya.cn/list.jsp?item=free&nextid=0&order=8&k=雾霾
    - 第二页是：
http://bbs.tianya.cn/list.jsp?item=free&nextid=1&order=8&k=雾霾


***
# 数据抓取：
> # 抓取天涯回帖网络
***

王成军

wangchengjun@nju.edu.cn

计算传播网 http://computational-communication.com

In [2]:
from IPython.display import display_html, HTML
HTML('<iframe src= http://bbs.tianya.cn/list.jsp?item=free&nextid=%d&order=8&k=PX width=1000 height=500></iframe>')
# the webpage we would like to crawl

In [3]:
page_num = 0
url = "http://bbs.tianya.cn/list.jsp?item=free&nextid=%d&order=8&k=PX"% page_num
content = requests.get(url).text 
soup = BeautifulSoup(content, "lxml") 
articles = soup.find_all('tr')
#td[0].find('a', {'class', ""})

In [4]:
articles[0]

<tr>
<th scope="col"> 标题</th>
<th scope="col">作者</th>
<th scope="col">点击</th>
<th scope="col">回复</th>
<th scope="col">发表时间</th>
</tr>

In [6]:
articles[1]

<tr class="bg">
<td class="td-title ">
<span class="face" title="">
</span>
<a href="/post-free-5043260-1.shtml" target="_blank">
							【吴钩一言堂】漳州PX成“二踢响”,说明老百姓科学素质不低<span class="art-ico art-ico-3" title="内有0张图片"></span>
</a>
</td>
<td><a class="author" href="http://www.tianya.cn/36959960" target="_blank">挑灯看吴钩</a></td>
<td>45821</td>
<td>219</td>
<td title="2015-04-07 21:30">04-07 21:30</td>
</tr>

In [7]:
len(articles[1:])

80

![](./img/inspect.png)

http://bbs.tianya.cn/list.jsp?item=free&nextid=0&order=8&k=PX
    
# 通过分析帖子列表的源代码，使用inspect方法，会发现所有要解析的内容都在‘td’这个层级下
    

In [10]:
for t in articles[1].find_all('td'): 
    print(t)

<td class="td-title ">
<span class="face" title="">
</span>
<a href="/post-free-5043260-1.shtml" target="_blank">
							【吴钩一言堂】漳州PX成“二踢响”,说明老百姓科学素质不低<span class="art-ico art-ico-3" title="内有0张图片"></span>
</a>
</td>
<td><a class="author" href="http://www.tianya.cn/36959960" target="_blank">挑灯看吴钩</a></td>
<td>45821</td>
<td>219</td>
<td title="2015-04-07 21:30">04-07 21:30</td>


In [11]:
td = articles[1].find_all('td')

In [27]:
td

[<td class="td-title ">
 <span class="face" title="">
 </span>
 <a href="/post-free-5043260-1.shtml" target="_blank">
 							【吴钩一言堂】漳州PX成“二踢响”,说明老百姓科学素质不低<span class="art-ico art-ico-3" title="内有0张图片"></span>
 </a>
 </td>,
 <td><a class="author" href="http://www.tianya.cn/36959960" target="_blank">挑灯看吴钩</a></td>,
 <td>45821</td>,
 <td>219</td>,
 <td title="2015-04-07 21:30">04-07 21:30</td>]

In [12]:
td[0]

<td class="td-title ">
<span class="face" title="">
</span>
<a href="/post-free-5043260-1.shtml" target="_blank">
							【吴钩一言堂】漳州PX成“二踢响”,说明老百姓科学素质不低<span class="art-ico art-ico-3" title="内有0张图片"></span>
</a>
</td>

In [13]:
td[0].text

'\n\n\n\r\n\t\t\t\t\t\t\t【吴钩一言堂】漳州PX成“二踢响”,说明老百姓科学素质不低\n\n'

In [14]:
td[0].text.strip()

'【吴钩一言堂】漳州PX成“二踢响”,说明老百姓科学素质不低'

In [15]:
td[0].a['href']

'/post-free-5043260-1.shtml'

In [21]:
td[1]

<td><a class="author" href="http://www.tianya.cn/36959960" target="_blank">挑灯看吴钩</a></td>

In [23]:
td[1].find('a', {'class', "author"})['href']

'http://www.tianya.cn/36959960'

In [28]:
td[2]

<td>45821</td>

In [29]:
td[3]

<td>219</td>

In [30]:
td[4]

<td title="2015-04-07 21:30">04-07 21:30</td>

In [32]:
records = []
for k, i in enumerate(articles[1:]):
    td = i.find_all('td')
    title = td[0].text.strip()
    try:
        title_url = td[0].find('a', {'class', "author"})['href'] 
    except:
        title_url = td[0].a['href']
    author = td[1].text
    author_url = td[1].a['href']
    views = td[2].text
    replies = td[3].text
    date = td[4]['title']
    record = '\t'.join([title, title_url, author, author_url, views, replies, date])
#     record = title + '\t' + title_url+ '\t' + author + 
#     '\t'+ author_url + '\t' + views+ '\t'  + replies+ '\t'+ date
    records.append(record)

In [34]:
records[:3]

['【吴钩一言堂】漳州PX成“二踢响”,说明老百姓科学素质不低\t/post-free-5043260-1.shtml\t挑灯看吴钩\thttp://www.tianya.cn/36959960\t45821\t219\t2015-04-07 21:30',
 '看到今天茂名PX事件的新闻通稿,我实不能忍\t/post-free-4239726-1.shtml\t专用马甲122\thttp://www.tianya.cn/16142449\t30515\t115\t2014-04-03 17:05',
 '【民间语文423期】 权力后遗症下难解的PX魔咒\t/post-free-5046436-1.shtml\t贾也\thttp://www.tianya.cn/50499450\t27410\t122\t2015-04-09 17:39']

In [35]:
print(records[2])

【民间语文423期】 权力后遗症下难解的PX魔咒	/post-free-5046436-1.shtml	贾也	http://www.tianya.cn/50499450	27410	122	2015-04-09 17:39


# 抓取天涯论坛PX帖子列表

回帖网络（Thread network）的结构
- 列表
- 主帖
- 回帖

In [45]:
articles[2]

<tr>
<td class="td-title facediamond">
<a class="diamond" href="http://bei.tianya.cn/wallet/zuan.do" target="_blank" title="钻石榜"></a>
<a class="" href="http://bbs.tianya.cn/post-free-5881411-1.shtml" target="_blank">检举揭发现任中国石油天然气集团公司董事长、党组书记王宜林行贿受贿</a>
</td>
<td><a class="author" href="http://www.tianya.cn/135764138" target="_blank">stevenxie871130</a></td>
<td>5591</td>
<td>34</td>
<td title="null">04-03 11:24</td>
</tr>

In [36]:
def crawler(page_num, file_name):
    try:
        # open the browser
        url = "http://bbs.tianya.cn/list.jsp?item=free&nextid=%d&order=8&k=PX" % page_num
        content = requests.get(url).text #获取网页的html文本
        soup = BeautifulSoup(content, "lxml") 
        articles = soup.find_all('tr')
        # write down info
        for i in articles[1:]:
            td = i.find_all('td')
            title = td[0].text.strip()
            title_url = td[0].a['href']
            author = td[1].text
            author_url = td[1].a['href']
            views = td[2].text
            replies = td[3].text
            date = td[4]['title']
            record = title + '\t' + title_url+ '\t' + author + '\t'+ \
                        author_url + '\t' + views+ '\t'  + replies+ '\t'+ date
            with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                        p.write(record+"\n") ##!!encode here to utf-8 to avoid encoding

    except Exception as e:
        print(e)
        pass

In [37]:
# crawl all pages
for page_num in range(10):
    print(page_num) 
    crawler(page_num, '../data/tianya_bbs_threads_list2018.txt') 

0
1
2
3
4
5
6
7
8
9


In [38]:
import pandas as pd

df = pd.read_csv('../data/tianya_bbs_threads_list2018.txt',
                 sep = "\t",  names = ['title', 'link', 'author', \
                 'author_page', 'click', 'reply', 'time'])
df[:3] 

,title,link,author,author_page,click,reply,time
0,"【吴钩一言堂】漳州PX成“二踢响”,说明老百姓科学素质不低",/post-free-5043260-1.shtml,挑灯看吴钩,http://www.tianya.cn/36959960,45821,219,2015-04-07 21:30
1,"看到今天茂名PX事件的新闻通稿,我实不能忍",/post-free-4239726-1.shtml,专用马甲122,http://www.tianya.cn/16142449,30515,115,2014-04-03 17:05
2,【民间语文423期】 权力后遗症下难解的PX魔咒,/post-free-5046436-1.shtml,贾也,http://www.tianya.cn/50499450,27410,122,2015-04-09 17:39


In [39]:
len(df)

437

In [40]:
len(df.link)

437

# 抓取作者信息

In [41]:
df.author_page[:5]

0    http://www.tianya.cn/36959960
1    http://www.tianya.cn/16142449
2    http://www.tianya.cn/50499450
3    http://www.tianya.cn/61943225
4    http://www.tianya.cn/25210087
Name: author_page, dtype: object

http://www.tianya.cn/62237033
    
http://www.tianya.cn/67896263
    

In [42]:
# user_info

In [43]:
url = df.author_page[10]
content = requests.get(url).text #获取网页的html文本
soup = BeautifulSoup(content, "lxml") 

In [44]:
soup

<!DOCTYPE HTML>
<!--[if IE 6]><html class="ie6 lte9 lte8 lte7" lang="zh-cn"><![endif]--><!--[if IE 8]><html class="ie8 lte9 lte8" lang="zh-cn"><![endif]--><!--[if IE 9]><html class="ie9 lte9" lang="zh-cn"><![endif]--><!--[if IE 7]><html class="ie7 lte9 lte8 lte7" lang="zh-cn"><![endif]--><!--[if !(IE 6) | !(IE 7) | !(IE 8) | !(IE 9)  ]><!--><html lang="zh-cn"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<title>我同意你的意见_天涯社区</title>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="Thu, 01 Nov 2012 10:00:00 GMT" http-equiv="Expires"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<link href="http://static.tianyaui.com/global/ty/TY.css" rel="stylesheet" type="text/css"/>
<link href="http://static.tianyaui.com/global/my/web/static/css/main_da523da.css" rel="stylesheet" type="text/css"/>
<link href="http://static.tianyaui.com/global/my/web/static/css/t_c5f218e.css" rel="stylesheet" type="text/css"/>
<link href="http://static.tianyaui.com/favicon.ico" rel="shortcut ic

In [49]:
print(url)

http://www.tianya.cn/44156220


'http://www.tianya.cn/44156220'

In [45]:
user_info = soup.find('div',  {'class', 'userinfo'})('p')
score, reg_time = [i.get_text()[4:] for i in user_info]
print(score, reg_time )

link_info = soup.find_all('div', {'class', 'link-box'})
followed_num, fans_num = [i.a.text for i in link_info]
print(followed_num, fans_num)

6362 2010-11-01 09:49:00
1 85


In [51]:
#activity = soup.find_all('span', {'class', 'subtitle'})
#post_num, reply_num = [j.text[2:] for i in activity[:1] for j in i('a')]
#print(post_num, reply_num)
#activity

In [53]:
#activity[0]

In [54]:
link_info = soup.find_all('div', {'class', 'link-box'})
followed_num, fans_num = [i.a.text for i in link_info]
print(followed_num, fans_num)

1 85


In [55]:
link_info[0].a.text

'1'

In [56]:
# user_info = soup.find('div',  {'class', 'userinfo'})('p')
# user_infos = [i.get_text()[4:] for i in user_info]
            
def author_crawler(url, file_name):
    try:
        content = requests.get(url).text #获取网页的html文本
        soup = BeautifulSoup(content, "lxml")
        link_info = soup.find_all('div', {'class', 'link-box'})
        followed_num, fans_num = [i.a.text for i in link_info]
        try:
            activity = soup.find_all('span', {'class', 'subtitle'})
            post_num, reply_num = [j.text[2:] for i in activity[:1] for j in i('a')]
        except:
            post_num, reply_num = '1', '0'
        record =  '\t'.join([url, followed_num, fans_num, post_num, reply_num])
        with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                    p.write(record+"\n") ##!!encode here to utf-8 to avoid encoding

    except Exception as e:
        print(e, url)
        record =  '\t'.join([url, 'na', 'na', 'na', 'na'])
        with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                    p.write(record+"\n") ##!!encode here to utf-8 to avoid encoding
        pass

In [58]:
#soup

In [137]:
url = df.author_page[10]
content = requests.get(url).text #获取网页的html文本
soup = BeautifulSoup(content, "lxml")
link_info = soup.find_all('div', {'class', 'link-box'})
followed_num, fans_num = [i.a.text for i in link_info]
try:
    activity = soup.find_all('span', {'class', 'subtitle'})
    post_num, reply_num = [j.text[2:] for i in activity[:1] for j in i('a')]
except:
    post_num, reply_num = '1', '0'
record =  '\t'.join([url, followed_num, fans_num, post_num, reply_num])


In [74]:
import sys
def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()

In [61]:
import time, random
for k, url in enumerate(df.author_page[:15]):
    time.sleep(random.random()) # 天涯存在反抓取机制，需要降低抓取速度！
    #flushPrint(k)
    author_crawler(url, '../data/tianya_bbs_threads_author_info2018.txt') 

not enough values to unpack (expected 2, got 0) http://www.tianya.cn/36959960
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/16142449
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/50499450
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/61943225
not enough values to unpack (expected 2, got 0) http://www.tianya.cn/80982782


http://www.tianya.cn/50499450/follow  

还可抓取他们的关注列表和粉丝列表

***
***
# 数据抓取：
> # 使用Python抓取回帖
***
***

王成军

wangchengjun@nju.edu.cn

计算传播网 http://computational-communication.com

In [62]:
df.link[2]

'/post-free-5046436-1.shtml'

In [64]:
url = 'http://bbs.tianya.cn' + df.link[2]
print(url)

http://bbs.tianya.cn/post-free-5046436-1.shtml


In [65]:
from IPython.display import display_html, HTML
HTML('<iframe src=http://bbs.tianya.cn/post-free-2849477-1.shtml width=1000 height=500></iframe>')
# the webpage we would like to crawl

In [66]:
post = requests.get(url).text #获取网页的html文本  
post_soup = BeautifulSoup(post, "lxml") 
#articles = soup.find_all('tr')

In [67]:
print (post_soup.prettify())[:5000]

<!DOCTYPE HTML>
<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   【民间语文423期】权力后遗症下难解的PX魔咒_天涯杂谈_论坛_天涯社区
  </title>
  <meta content="【民间语文423期】权力后遗症下难解的PX魔咒主笔|贾也  官方微信号：tianyaguancha导语：燃爆引起的社会震撼据新华网福建漳州4月9日电，6日下午发生的漳州古雷石化大火经全力扑救，终于在9日凌晨2时57分被扑灭。目前，相关善后工作..." itemprop="description" name="description"/>
  <meta content="【民间语文423期】权力后遗症下难解的PX魔咒_天涯杂谈_论坛_天涯社区" itemprop="name"/>
  <meta content="" itemprop="image"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="贾也" name="author"/>
  <meta content="no-transform" http-equiv="Cache-Control"/>
  <meta content="no-siteapp" http-equiv="Cache-Control"/>
  <meta content="pc" name="applicable-device"/>
  <meta content="format=xhtml; url=http://bbs.tianya.cn/m/post-free-5046436-1.shtml" name="mobile-agent"/>
  <meta content="format=html5; url=http://bbs.tianya.cn/m/post-free-5046436-1.shtml" name="mobile-agent"/>
  <meta content="format=wml; url=http://bbs.tianya.cn/m/post-free-5046436-1.shtml" name="mobile-agent"/>
  <li

TypeError: 'NoneType' object is not subscriptable

In [68]:
pa = post_soup.find_all('div', {'class', 'atl-item'})
len(pa)

71

In [69]:
print(pa[0]) 

<div _host="%E8%B4%BE%E4%B9%9F" _hostid="50499450" class="atl-item host-item">
<div class="atl-content">
<div class="atl-con-hd clearfix">
<div class="atl-con-hd-l"></div>
<div class="atl-con-hd-r"></div>
</div>
<div class="atl-con-bd clearfix">
<div class="bbs-content clearfix">
<img original="http://img3.laibafile.cn/p/m/130429226.gif" src="http://static.tianyaui.com/img/static/2011/imgloading.gif" title="点击图片查看幻灯模式"/><br/>
<br/>
<b>【民间语文423期】权力后遗症下难解的PX魔咒</b><br/>
<br/>
<b>主笔|贾也  官方微信号：tianyaguancha</b>　<br/>
　<img original="http://img3.laibafile.cn/p/m/210142061.jpg" src="http://static.tianyaui.com/img/static/2011/imgloading.gif" title="点击图片查看幻灯模式"/><br/>
　　<b>导语：燃爆引起的社会震撼</b><br/>
　　<br/>
　　据新华网福建漳州4月9日电，6日下午发生的漳州古雷石化大火经全力扑救，终于在9日凌晨2时57分被扑灭。目前，相关善后工作正在有序进行中。火总算是扑灭了，但在人们心头燃起的怒火怨气却远未扑灭：从2007年福建厦门反PX上街散步开始，到现在人们谈PX色变，这次爆燃让多少PX科普打了水漂，而自此之后，PX项目将来推进的难度可想而知了。PX项目是好项目，为什么好项目面临如此境遇？问题究竟出在哪里？不妨让贾也来作一番深度的解读。<br/>
<br/>
　　<b>一、难解的PX魔咒</b><br/>
<br/>
　　PX用途很广，与我们日常生活息息相关，几乎渗透到衣、食、住、行方方面面，药用的胶

In [70]:
pa[1]

<div _host="烟火" _hostid="45961" class="atl-item" id="2" js_restime="2015-04-09 18:05:20" js_username="烟火" replyid="129085630">
<div class="atl-head" id="9d3c2b08f99f43961df9793406ab014c">
<div class="atl-head-reply"></div>
<div class="atl-info">
<span>作者：<a class="js-vip-check" href="http://www.tianya.cn/45961" target="_blank" uid="45961" uname="烟火">烟火</a> </span>
<span>时间：2015-04-09 18:05:20</span>
</div>
</div>
<div class="atl-content">
<div class="atl-con-hd clearfix">
<div class="atl-con-hd-l"></div>
<div class="atl-con-hd-r"></div>
</div>
<div class="atl-con-bd clearfix">
<div class="bbs-content">
							　　沙发！
							
							
						</div>
<div class="atl-reply">
<a author="烟火" authorid="45961" class="reportme a-link" href="javascript:void(0);" replyid="129085630" replytime="2015-04-09 18:05:20">举报</a> |
														
							<span>2楼</span> |
							<a class="a-link-2 ir-shang" floor="2" href="javascript:void(0);" title="打赏层主">
								打赏
							</a> |
							<a class="a-link-2 re

In [71]:
pa[70]

<div _host="反光的镜框" _hostid="90925145" class="atl-item" id="100" js_restime="2015-04-13 09:31:06" js_username="反光的镜框" replyid="129085841">
<div class="atl-head" id="6020994e96f2964d6ca6bf39e0ad9d21">
<div class="atl-head-reply"></div>
<div class="atl-info">
<span>作者：<a class="js-vip-check" href="http://www.tianya.cn/90925145" target="_blank" uid="90925145" uname="反光的镜框">反光的镜框</a> </span>
<span>时间：2015-04-13 09:31:06</span>
</div>
</div>
<div class="atl-content">
<div class="atl-con-hd clearfix">
<div class="atl-con-hd-l"></div>
<div class="atl-con-hd-r"></div>
</div>
<div class="atl-con-bd clearfix">
<div class="bbs-content">
							　　好文章。
							
							
						</div>
<div class="atl-reply">
<a author="反光的镜框" authorid="90925145" class="reportme a-link" href="javascript:void(0);" replyid="129085841" replytime="2015-04-13 09:31:06">举报</a> |
														
							<span>100楼</span> |
							<a class="a-link-2 ir-shang" floor="100" href="javascript:void(0);" title="打赏层主">
								打赏
							<

作者：柠檬在追逐 时间：2012-10-28 21:33:55

　　@lice5 2012-10-28 20:37:17
  
　　作为宁波人 还是说一句：革命尚未成功 同志仍需努力 
  
　　-----------------------------
  
　　对 现在说成功还太乐观，就怕说一套做一套

作者：lice5 时间：2012-10-28 20:37:17
        
　　作为宁波人 还是说一句：革命尚未成功 同志仍需努力

4	/post-free-4242156-1.shtml	2014-04-09 15:55:35	61943225	野渡自渡人	@Y雷政府34楼2014-04-0422:30:34　　野渡君雄文！支持是必须的。　　-----------------------------　　@清坪过客16楼2014-04-0804:09:48　　绝对的权力导致绝对的腐败！　　-----------------------------　　@T大漠鱼T35楼2014-04-0810:17:27　　@周丕东@普欣@拾月霜寒2012@小摸包@姚文嚼字@四號@凌宸@乔志峰@野渡自渡人@曾兵2010@缠绕夜色@曾颖@风青扬请关注

In [233]:
pa[0].find('div', {'class', 'bbs-content'}).text.strip()

'.marg {width:100%; background:#E8E3CF; overflow:hidden; margin:0 auto; padding-top:10px; padding-left:0px;padding-right: 0px;padding-bottom:10px;}.ztpage {BORDER-RIGHT: #dcc696 3px solid; PADDING-RIGHT: 5px; BORDER-TOP: #dcc696 3px solid; PADDING-LEFT: 5px; PADDING-BOTTOM: 5px; MARGIN: 0px auto; BORDER-LEFT: #dcc696 3px solid; WIDTH: 880px; PADDING-TOP: 5px; BORDER-BOTTOM: #dcc696 3px solid; BACKGROUND-COLOR: #dededd;}.ywdu{width:880px;}.ywdu .dao-1{float:left;width:500px; height: 370px}.ywdu .dao-1t{font-size:18px; font-family:微软雅黑; color:#ff0000;padding:6px 15px;float:left;width:460px;padding-left:215px; padding-top:18px;padding-right:2px}.ywdu .dao-1m{font-size:25px; font-family:微软雅黑,黑体; color:#90060B;font-weight:bold; padding:6px 15px;float:left;width:460px;padding-left:30px; padding-top:40px;padding-right:10px}.ywdu .dao-1b{font-size:14px; font-family:微软雅黑,黑体; color:#fff; line-height:16px;float:left;width:460px;height: 238px;padding-left:10px; padding-top:30px;padding-right:180px

In [76]:
print(pa[0].text.strip())#.find('div', {'class', 'marg'}).text.strip()

【民间语文423期】权力后遗症下难解的PX魔咒

主笔|贾也  官方微信号：tianyaguancha　
　
　　导语：燃爆引起的社会震撼
　　
　　据新华网福建漳州4月9日电，6日下午发生的漳州古雷石化大火经全力扑救，终于在9日凌晨2时57分被扑灭。目前，相关善后工作正在有序进行中。火总算是扑灭了，但在人们心头燃起的怒火怨气却远未扑灭：从2007年福建厦门反PX上街散步开始，到现在人们谈PX色变，这次爆燃让多少PX科普打了水漂，而自此之后，PX项目将来推进的难度可想而知了。PX项目是好项目，为什么好项目面临如此境遇？问题究竟出在哪里？不妨让贾也来作一番深度的解读。

　　一、难解的PX魔咒

　　PX用途很广，与我们日常生活息息相关，几乎渗透到衣、食、住、行方方面面，药用的胶囊、食用的塑料瓶、穿的衣服，乃至装修用的涂料，主要的原料都是PX。特别是PX最重要的下游产品PET，正在越来越多地取代铝、玻璃、陶瓷、纸张等材料。而且论“毒性”，还“屈居”食盐之后，绝对属于低毒产品。
　　可以这么说，PX是个好东西，技术也没问题，产品也是民生必须，利润也丰厚，中国没理由不上，正因为如此，政府即使冒着民众上街散步这等天下之大韪的事，也要想让PX项目上马。

　　政府打着如意算盘，想尽一切办法想让PX项目上马，民众却不买它的账，纷纷来拆政府的台。就像福建的PX项目，当初在厦门受到了民众的强烈抵制，这难道有“境外势力”作祟，有“不法分子”鼓惑？不要总把民众当作三岁乳儿啊，污辱这么一大片人的智商，动不动就说他们被人煽动、利用了——这种定性从来就是蛮不讲理的。

　　民众确实有些“不明真相”，究其原因，政府还是要返身于于己，找自身原因的。政府向来喜欢暗箱操作，办事不阳光、不透明，跟企业又走得太近，尽玩些“特事特办”的“双人转”游戏，合作协议签订了，工程上马政府比企业都要猴急，又想着绕开民众的监督，对民众只是象征性地向公示一下，尽捡些正面的东西来宣传：PX是安全无毒的，PX项目能带动地方经济……

　　问题是，老百姓一开始压根就不知道PX是什么？而政府呢，又不想花时间和精力向民众释疑解惑。无奈之下，民众只能DIY起PX的科普知识了。结果他们发现跟政府透露出来的信息根本不对称：PX有没有毒？不管低毒不低毒，反正是有毒的；PX会不会致癌？缺乏证据，缺乏证据就是不排除致癌；最让民众担忧的还是安

In [77]:
pa[67].find('div', {'class', 'bbs-content'}).text.strip()

'写的很好，希望中国挺过难关，痛定思痛，更希望有更多像楼主这样有真知灼见的人为我国献计献策，走向光明。'

In [78]:
pa[1].a 

<a class="js-vip-check" href="http://www.tianya.cn/45961" target="_blank" uid="45961" uname="烟火">烟火</a>

In [79]:
pa[0].find('a', class_ = 'reportme a-link')

<a author="贾也" authorid="50499450" class="reportme a-link" href="javascript:void(0);" replyid="0" replytime="2015-04-09 17:39:00"> 举报</a>

In [80]:
pa[0].find('a', class_ = 'reportme a-link')['replytime']

'2015-04-09 17:39:00'

In [81]:
pa[0].find('a', class_ = 'reportme a-link')['author']

'贾也'

In [82]:
for i in pa[:10]:
    p_info = i.find('a', class_ = 'reportme a-link')
    p_time = p_info['replytime']
    p_author_id = p_info['authorid']
    p_author_name = p_info['author']
    p_content = i.find('div', {'class', 'bbs-content'}).text.strip()
    p_content = p_content.replace('\t', '')
    print(p_time, '--->', p_author_id, '--->', p_author_name,'--->', p_content, '\n')

2015-04-09 17:39:00 ---> 50499450 ---> 贾也 ---> 【民间语文423期】权力后遗症下难解的PX魔咒

主笔|贾也  官方微信号：tianyaguancha　
　
　　导语：燃爆引起的社会震撼
　　
　　据新华网福建漳州4月9日电，6日下午发生的漳州古雷石化大火经全力扑救，终于在9日凌晨2时57分被扑灭。目前，相关善后工作正在有序进行中。火总算是扑灭了，但在人们心头燃起的怒火怨气却远未扑灭：从2007年福建厦门反PX上街散步开始，到现在人们谈PX色变，这次爆燃让多少PX科普打了水漂，而自此之后，PX项目将来推进的难度可想而知了。PX项目是好项目，为什么好项目面临如此境遇？问题究竟出在哪里？不妨让贾也来作一番深度的解读。

　　一、难解的PX魔咒

　　PX用途很广，与我们日常生活息息相关，几乎渗透到衣、食、住、行方方面面，药用的胶囊、食用的塑料瓶、穿的衣服，乃至装修用的涂料，主要的原料都是PX。特别是PX最重要的下游产品PET，正在越来越多地取代铝、玻璃、陶瓷、纸张等材料。而且论“毒性”，还“屈居”食盐之后，绝对属于低毒产品。
　　可以这么说，PX是个好东西，技术也没问题，产品也是民生必须，利润也丰厚，中国没理由不上，正因为如此，政府即使冒着民众上街散步这等天下之大韪的事，也要想让PX项目上马。

　　政府打着如意算盘，想尽一切办法想让PX项目上马，民众却不买它的账，纷纷来拆政府的台。就像福建的PX项目，当初在厦门受到了民众的强烈抵制，这难道有“境外势力”作祟，有“不法分子”鼓惑？不要总把民众当作三岁乳儿啊，污辱这么一大片人的智商，动不动就说他们被人煽动、利用了——这种定性从来就是蛮不讲理的。

　　民众确实有些“不明真相”，究其原因，政府还是要返身于于己，找自身原因的。政府向来喜欢暗箱操作，办事不阳光、不透明，跟企业又走得太近，尽玩些“特事特办”的“双人转”游戏，合作协议签订了，工程上马政府比企业都要猴急，又想着绕开民众的监督，对民众只是象征性地向公示一下，尽捡些正面的东西来宣传：PX是安全无毒的，PX项目能带动地方经济……

　　问题是，老百姓一开始压根就不知道PX是什么？而政府呢，又不想花时间和精力向民众释疑解惑。无奈之下，民众只能DIY起PX的科普知识了。结果他们发现跟政府透露出来的信息根本不对称：PX有没有毒？不管低

# 如何翻页

http://bbs.tianya.cn/post-free-2848797-1.shtml
    
    http://bbs.tianya.cn/post-free-2848797-2.shtml
        
        http://bbs.tianya.cn/post-free-2848797-3.shtml

In [83]:
post_soup.find('div', {'class', 'atl-pages'})#.['onsubmit']

<div class="atl-pages"><form action="" method="get" onsubmit="return goPage(this,'free',5046436,2);">
<span>上页</span>
<strong>1</strong>
<a href="/post-free-5046436-2.shtml">2</a>
<a class="js-keyboard-next" href="/post-free-5046436-2.shtml">下页</a>
 到<input class="pagetxt" name="page" type="text"/>页 <input class="pagebtn" maxlength="6" name="gopage" type="submit" value="确定"/></form></div>

In [84]:
post_pages = post_soup.find('div', {'class', 'atl-pages'})
post_pages = post_pages.form['onsubmit'].split(',')[-1].split(')')[0]
post_pages
#post_soup.select('.atl-pages')[0].select('form')[0].select('onsubmit')


'2'

In [85]:
url = 'http://bbs.tianya.cn' + df.link[2]
url_base = ''.join(url.split('-')[:-1]) + '-%d.shtml'
url_base

'http://bbs.tianya.cn/postfree5046436-%d.shtml'

In [86]:
def parsePage(pa):
    records = []
    for i in pa:
        p_info = i.find('a', class_ = 'reportme a-link')
        p_time = p_info['replytime']
        p_author_id = p_info['authorid']
        p_author_name = p_info['author']
        p_content = i.find('div', {'class', 'bbs-content'}).text.strip()
        p_content = p_content.replace('\t', '').replace('\n', '')#.replace(' ', '')
        record = p_time + '\t' + p_author_id+ '\t' + p_author_name + '\t'+ p_content
        records.append(record)
    return records

import sys
def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()

In [87]:
url_1 = 'http://bbs.tianya.cn' + df.link[10]
content = requests.get(url_1).text #获取网页的html文本
post_soup = BeautifulSoup(content, "lxml") 
pa = post_soup.find_all('div', {'class', 'atl-item'})
b = post_soup.find('div', class_= 'atl-pages')
b

<div class="atl-pages host-pages"></div>

In [88]:
url_0 = 'http://bbs.tianya.cn' + df.link[2]
content = requests.get(url_0).text #获取网页的html文本
post_soup = BeautifulSoup(content, "lxml") 
pa = post_soup.find_all('div', {'class', 'atl-item'})
a = post_soup.find('div', class_= 'atl-pages')
a

<div class="atl-pages"><form action="" method="get" onsubmit="return goPage(this,'free',5046436,2);">
<span>上页</span>
<strong>1</strong>
<a href="/post-free-5046436-2.shtml">2</a>
<a class="js-keyboard-next" href="/post-free-5046436-2.shtml">下页</a>
 到<input class="pagetxt" name="page" type="text"/>页 <input class="pagebtn" maxlength="6" name="gopage" type="submit" value="确定"/></form></div>

In [89]:
a.form

<form action="" method="get" onsubmit="return goPage(this,'free',5046436,2);">
<span>上页</span>
<strong>1</strong>
<a href="/post-free-5046436-2.shtml">2</a>
<a class="js-keyboard-next" href="/post-free-5046436-2.shtml">下页</a>
 到<input class="pagetxt" name="page" type="text"/>页 <input class="pagebtn" maxlength="6" name="gopage" type="submit" value="确定"/></form>

In [90]:
if b.form:
    print('true')
else:
    print('false')

false


In [91]:
import random
import time

def crawler(url, file_name):
    try:
        # open the browser
        url_1 = 'http://bbs.tianya.cn' + url
        content = requests.get(url_0).text #获取网页的html文本
        post_soup = BeautifulSoup(content, "lxml") 
        # how many pages in a post
        post_form = post_soup.find('div', {'class', 'atl-pages'})
        if post_form.form:
            post_pages = post_form.form['onsubmit'].split(',')[-1].split(')')[0]
            post_pages = int(post_pages)
            url_base = '-'.join(url_1.split('-')[:-1]) + '-%d.shtml'
        else:
            post_pages = 1
        # for the first page
        pa = post_soup.find_all('div', {'class', 'atl-item'})
        records = parsePage(pa)
        with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
            for record in records:    
                p.write('1'+ '\t' + url + '\t' + record+"\n") 
        # for the 2nd+ pages
        if post_pages > 1:
            for page_num in range(2, post_pages+1):
                time.sleep(random.random())
                flushPrint(page_num)
                url2 =url_base  % page_num
                content = requests.get(url2).text #获取网页的html文本
                post_soup = BeautifulSoup(content, "lxml") 
                pa = post_soup.find_all('div', {'class', 'atl-item'})
                records = parsePage(pa)
                with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                    for record in records:    
                        p.write(str(page_num) + '\t' +url + '\t' + record+"\n") 
        else:
            pass
    except Exception as e:
        print(e)
        pass

# 测试

In [92]:
url = df.link[2]
file_name = '../data/tianya_bbs_threads_2018test.txt'
crawler(url, file_name)

2

# 正式抓取！

In [94]:
for k, link in enumerate(df.link):
    flushPrint(link)
    if k % 10== 0:
        print('This it the post of : ' + str(k))
    file_name = '../data/tianya_bbs_threads_network_2018.txt'
    crawler(link, file_name)

/post-free-5043260-1.shtmlThis it the post of : 0
/post-free-2856201-1.shtmlThis it the post of : 10
/post-free-923387-1.shtmllThis it the post of : 20
/post-free-5046982-1.shtmlThis it the post of : 30
/post-free-5054821-1.shtmlThis it the post of : 40
/post-free-3373483-1.shtmlThis it the post of : 50
/post-free-5041433-1.shtmlThis it the post of : 60
/post-free-3835452-1.shtmlThis it the post of : 70
/post-free-2241144-1.shtmlThis it the post of : 80
/post-free-1066300-1.shtmlThis it the post of : 90
/post-free-3287926-1.shtmlThis it the post of : 100
/post-free-929110-1.shtmllThis it the post of : 110
/post-free-3358276-1.shtmlThis it the post of : 120
/post-free-3281916-1.shtmlThis it the post of : 130
/post-free-3328954-1.shtmlThis it the post of : 140
/post-free-3312023-1.shtmlThis it the post of : 150
/post-free-3833509-1.shtmlThis it the post of : 160
/post-free-3833373-1.shtmlThis it the post of : 170
/post-free-3291155-1.shtmlThis it the post of : 180
/post-free-3302091-1.sh

# 读取数据

In [95]:
dtt = []
with open('../data/tianya_bbs_threads_network_2018.txt', 'r') as f:
    for line in f:
        pnum, link, time, author_id, author, content = line.replace('\n', '').split('\t')
        dtt.append([pnum, link, time, author_id, author, content])
len(dtt)

33442

In [96]:
dt = pd.DataFrame(dtt)
dt[:5]

,0,1,2,3,4,5
0,1,/post-free-5043260-1.shtml,2015-04-09 17:39:00,50499450,贾也,【民间语文423期】权力后遗症下难解的PX魔咒主笔|贾也 官方微信号：tianyaguan...
1,1,/post-free-5043260-1.shtml,2015-04-09 18:05:20,45961,烟火,沙发！
2,1,/post-free-5043260-1.shtml,2015-04-09 18:14:39,96044178,chunge2014,好文，特别是后面部分分析很到位。
3,1,/post-free-5043260-1.shtml,2015-04-10 09:47:17,98914479,扯蛋的智商,@贾也 每个人的想法都不同，层次决定眼界。 发展才是硬道理。 理解不了不怪你，怪基因...
4,1,/post-free-5043260-1.shtml,2015-04-10 10:06:51,11287309,骑马蹲裆,政府做事情就喜欢暗箱操作，比如我家门前这条路，突然就通知要挖路，事前从没有征求居民的意见，更...


In [97]:
dt=dt.rename(columns = {0:'page_num', 1:'link', 2:'time', 3:'author',4:'author_name', 5:'reply'})
dt[:5]

,page_num,link,time,author,author_name,reply
0,1,/post-free-5043260-1.shtml,2015-04-09 17:39:00,50499450,贾也,【民间语文423期】权力后遗症下难解的PX魔咒主笔|贾也 官方微信号：tianyaguan...
1,1,/post-free-5043260-1.shtml,2015-04-09 18:05:20,45961,烟火,沙发！
2,1,/post-free-5043260-1.shtml,2015-04-09 18:14:39,96044178,chunge2014,好文，特别是后面部分分析很到位。
3,1,/post-free-5043260-1.shtml,2015-04-10 09:47:17,98914479,扯蛋的智商,@贾也 每个人的想法都不同，层次决定眼界。 发展才是硬道理。 理解不了不怪你，怪基因...
4,1,/post-free-5043260-1.shtml,2015-04-10 10:06:51,11287309,骑马蹲裆,政府做事情就喜欢暗箱操作，比如我家门前这条路，突然就通知要挖路，事前从没有征求居民的意见，更...


In [98]:
dt.reply[:100]

0     【民间语文423期】权力后遗症下难解的PX魔咒主笔|贾也  官方微信号：tianyaguan...
1                                                   沙发！
2                                      好文，特别是后面部分分析很到位。
3     @贾也 　　每个人的想法都不同，层次决定眼界。　　发展才是硬道理。　　理解不了不怪你，怪基因...
4     政府做事情就喜欢暗箱操作，比如我家门前这条路，突然就通知要挖路，事前从没有征求居民的意见，更...
5                                                 威武，支持
6                     写得很好，对比现在众多批评PX项目的文章，唯一能让我心悦诚服的。。
7     @扯蛋的智商 2015-04-10 09:47:17　　@贾也　　每个人的想法都不同，层次决...
8     请求帮助！！我在淘宝卖了一块太岁肉灵芝，9500元，交易前就和买家讲好此物品特殊性不能退换，...
9                                     民众不高兴，谁都别想高兴。说得好。
10    分析得太到位了，真的是入木三分！“一场爆燃，震撼人心，震撼到不止于一个PX项目，更是我们的这...
11                                                     
12                                         PX原来只是一个导火索。
13                                         @贾也 中国就是这个样子
14                                    月关的依法治国还有很长的路要走啊！
15                                             三自信哪里去了？
16                                                 说得好！
17    @扯蛋的智商  4楼  2015-04-10 09:47:00　　@贾也 　　每个人

## 总帖数是多少？

http://search.tianya.cn/bbs?q=PX  共有18459 条内容

In [93]:
18459/50

369

实际上到第10页就没有了 http://bbs.tianya.cn/list.jsp?item=free&order=1&nextid=9&k=PX, 原来那只是天涯论坛，还有其它各种版块，如天涯聚焦：  http://focus.tianya.cn/  等等。

- 娱乐八卦 512
- 股市论坛 187
- 情感天地 242
- 天涯杂谈 1768 

在天涯杂谈搜索雾霾，有41页 http://bbs.tianya.cn/list.jsp?item=free&order=20&nextid=40&k=%E9%9B%BE%E9%9C%BE

# 天涯SDK

http://open.tianya.cn/wiki/index.php?title=SDK%E4%B8%8B%E8%BD%BD